# COGS 108 - Data Checkpoint

## Authors

Example team list and credits:
- Shaila Valenzuela : dataset cleaning
- Nicholas Nurwinata : dataset cleaning
- Khuyen Lai
- Weng Lok

## Research Question

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback



## Background and Prior Work

Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback

## Hypothesis


Instructions: REPLACE the contents of this cell with your work, including any updates to recover points lost in your proposal feedback


## Data

### Data overview

Instructions: REPLACE the contents of this cell with descriptions of your actual datasets.

For each dataset include the following information
- Dataset #1
  - Dataset Name:
  - Link to the dataset:
  - Number of observations:
  - Number of variables:
  - Description of the variables most relevant to this project
  - Descriptions of any shortcomings this dataset has with repsect to the project
- Dataset #2 (if you have more than one!)
  - same as above
- etc

Each dataset deserves either a set of bullet points as above or a few sentences if you prefer that method.

If you plan to use multiple datasets, add a few sentences about how you plan to combine these datasets.

In [59]:
# Run this code every time when you're actively developing modules in .py files.  It's not needed if you aren't making modules
#
## this code is necessary for making sure that any modules we load are updated here 
## when their source code .py files are modified

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
# Setup code -- this only needs to be run once after cloning the repo!
# this code downloads the data from its source to the `data/00-raw/` directory
# if the data hasn't updated you don't need to do this again!

# if you don't already have these packages (you should!) uncomment this line
# %pip install requests tqdm

import sys
sys.path.append('./modules') # this tells python where to look for modules to import

import get_data # this is where we get the function we need to download data

# replace the urls and filenames in this list with your actual datafiles
# yes you can use Google drive share links or whatever
# format is a list of dictionaries; 
# each dict has keys of 
#   'url' where the resource is located
#   'filename' for the local filename where it will be stored 
datafiles = [
    {
        "url": "https://drive.google.com/uc?export=download&id=1jv0r6OsBZyt-nr2Sb9jtaAaxqI-OD0fP",
        "filename": "student-performance.csv",
    },
    {
        "url": "https://drive.google.com/uc?export=download&id=130iTt9KYDazxSX8UP-FCwq02D95auli2",
        "filename": "ai-impact-students-performance-dataset.csv",
    },
]


get_data.get_raw(datafiles,destination_directory='data/00-raw/')

Overall Download Progress:   0%|          | 0/2 [00:00<?, ?it/s]

Overall Download Progress:  50%|█████     | 1/2 [00:01<00:01,  1.43s/it]

Successfully downloaded: student-performance.csv


Overall Download Progress: 100%|██████████| 2/2 [00:02<00:00,  1.45s/it]

Successfully downloaded: ai-impact-students-performance-dataset.csv


In [61]:
import pandas as pd
import numpy as np

## Dataset #1

### Student Performance Dataset

**Source:** Kaggle – Student_performance_data  
https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset

This dataset contains individual-level student records describing demographic characteristics, study habits, parental involvement, extracurricular participation, and academic performance. Each row represents one student, identified by a unique `StudentID` ranging from 1001 to 3392. The dataset includes approximately **2,392 student observations** and **15 variables**.

---

### Outcome Variable

The primary outcome variable for this project is:

- `GPA` — Grade Point Average on a scale from **2.0 to 4.0**. Higher values indicate stronger academic performance. GPA is influenced by study habits, parental involvement, and extracurricular engagement.

Although the dataset also includes `GradeClass`, which categorizes GPA into letter grades (A–F), we will model `GPA` as a **continuous variable** for regression analysis. Since `GradeClass` is derived directly from `GPA`, it will not be used as a predictor to avoid target leakage.

---

### Demographic Variables

These variables allow us to examine socioeconomic and demographic influences on academic performance:

- `Age` — 15–18 years  
- `Gender` — Binary encoded (0 = Male, 1 = Female)  
- `Ethnicity` — 0 = Caucasian, 1 = African American, 2 = Asian, 3 = Other  
- `ParentalEducation` — Highest parental education level:  
  - 0 = None  
  - 1 = High School  
  - 2 = Some College  
  - 3 = Bachelor’s  
  - 4 = Higher  

---

### Study Habits

These variables directly relate to our hypothesis that stronger academic habits predict higher GPA:

- `StudyTimeWeekly` — Weekly study time in hours (0–20 hours)  
- `Absences` — Number of absences during the school year (0–30)  
- `Tutoring` — Participation in tutoring (0 = No, 1 = Yes)  

---

### Parental Involvement

This variable evaluates whether family engagement contributes to student outcomes:

- `ParentalInvolvement` —  
  - 0 = None  
  - 1 = Low  
  - 2 = Moderate  
  - 3 = High  
  - 4 = Very High  

---

### Extracurricular Activities

These variables allow us to examine whether time spent outside academics relates to GPA. Each variable is binary (0 = No, 1 = Yes):

- `Extracurricular`  
- `Sports`  
- `Music`  
- `Volunteering`  

---

### Dataset Limitations & Concerns

- **Age Range (15–18 years):** The dataset appears to represent high school students rather than higher-education students. While our research question refers to higher education, this dataset still allows us to explore relationships between habits and academic performance in a structured academic environment.  

- **Self-Reported Behavior:** Study time and absences may be self-reported and subject to reporting bias.  

- **Categorical Encoding:** Several variables are numerically coded but represent categories. These must be treated as categorical variables during modeling to avoid incorrect assumptions of linear relationships.  

- **Derived Variable (`GradeClass`):** Because `GradeClass` is calculated directly from `GPA`, including both in modeling would introduce leakage. Therefore, `GPA` will be used as the regression outcome.

In [66]:
df_raw = pd.read_csv('data/00-raw/student-performance.csv')
df = df_raw.copy()

print("First 5 rows")
display(df.head())

#Is the dataset tidy?
print("\nDataset shape:")
print("Rows (observations):", df.shape[0])
print("Columns (variables):", df.shape[1])

print("\nColumn names:")
print(df.columns)

#Missing data Analysis
missing_counts = df.isna().sum()
missing_percent = df.isna().mean() * 100

missing_summary = pd.DataFrame({
    "Missing Count" : missing_counts,
    "Missing Percent" : missing_percent
})

print("\nMissing Data Summary:")
display(missing_summary)

# Suspicious Value Checks 
print("\nSummary statistics:")
display(df.describe())

print("\nChecking for impossible GPA values:")
display(df[(df["GPA"]<0) | (df["GPA"]>4)])

print("\nChecking for negative StudyTimeWeekly:")
display(df[df["StudyTimeWeekly"]<0])

print("\nChecking for negative Absences:")
display(df[df["Absences"]<0])

# Convert categorical vars
categorical_cols = [
    "Gender", "Ethnicity", "ParentalEducation",
    "Tutoring",  "ParentalSupport", 
    "Extracurricular", "Sports", "Music", "Volunteering",
    "GradeClass"
]

for col in categorical_cols:
    df[col] = df[col].astype("category")

df_clean = df.drop(columns=["GradeClass"])

df_raw.to_csv("data/01-interim/students_interim.csv", index=False)
df_clean.to_csv("data/02-processed/students_processed.csv", index=False)

print("\nFinal cleaned dataset shape:")
print(df_clean.shape)

First 5 rows


,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0



Dataset shape:
Rows (observations): 2392
Columns (variables): 15

Column names:
Index(['StudentID', 'Age', 'Gender', 'Ethnicity', 'ParentalEducation',
       'StudyTimeWeekly', 'Absences', 'Tutoring', 'ParentalSupport',
       'Extracurricular', 'Sports', 'Music', 'Volunteering', 'GPA',
       'GradeClass'],
      dtype='object')

Missing Data Summary:


,Missing Count,Missing Percent
StudentID,0,0.0
Age,0,0.0
Gender,0,0.0
Ethnicity,0,0.0
ParentalEducation,0,0.0
StudyTimeWeekly,0,0.0
Absences,0,0.0
Tutoring,0,0.0
ParentalSupport,0,0.0
Extracurricular,0,0.0



Summary statistics:


...


Checking for negative StudyTimeWeekly:


,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass



Checking for negative Absences:


,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass



Final cleaned dataset shape:
(2392, 14)


### Dataset #2 

Kaggle ai impact on students performance dataset

# Student Performance & Academic Trends Dataset  
## A Comprehensive Dataset Exploring How AI Usage Shapes Learning Outcomes  

This dataset contains individual-level student records examining how Artificial Intelligence (AI)–based learning tools influence academic performance. Each row represents one student, identified by a unique `student_id`. The dataset contains approximately **1,000 observations** and **26 variables**, capturing demographic characteristics, AI engagement patterns, study behavior, and academic outcomes.

---

## Outcome Variable  

The primary outcome variable for this project is:

**final_score** – Overall academic score combining exams, assignments, and participation. Higher values indicate stronger academic performance.

We will model `final_score` as a **continuous variable** for regression analysis.

The dataset also includes:

- **passed** – Binary indicator (1 = Passed, 0 = Failed)  
- **performance_category** – Categorized performance (Low, Medium, High)

Because both of these variables are derived directly from academic scoring metrics, they will not be used as predictors when modeling `final_score` to avoid target leakage.

---

## Demographic Variables  

These variables allow us to examine whether demographic characteristics influence AI usage and academic outcomes.

- **age** – Age of the student in years  
- **gender** – Gender of the student (Male, Female, Other)  
- **grade_level** – Academic year or grade level  

---

## AI Usage Variables  

These variables capture how students interact with AI-based learning systems.

- **uses_ai** – Binary indicator (1 = Yes, 0 = No)  
- **ai_usage_time_minutes** – Total time spent using AI tools (minutes)  
- **ai_tools_used** – Names of AI tools used (e.g., ChatGPT, Copilot)  
- **ai_usage_purpose** – Purpose of AI use (Exam Prep, Notes, Coding, etc.)  
- **ai_dependency_score** – Reliance on AI tools (higher = more dependent)  
- **ai_generated_content_percentage** – Percentage of assignments generated using AI  
- **ai_prompts_per_week** – Number of AI prompts submitted weekly  
- **ai_ethics_score** – Score reflecting ethical AI usage  

These variables allow us to examine whether AI engagement improves performance, substitutes independent learning, or increases dependency.

---

## Study Habits & Behavioral Variables  

These variables reflect general academic behavior and lifestyle factors.

- **study_hours_per_day** – Average daily study time  
- **study_consistency_index** – Consistency of study habits (higher = more consistent)  
- **concept_understanding_score** – Understanding of key concepts  
- **tutoring_hours** – Hours spent in extra tutoring  
- **class_participation_score** – Level of classroom engagement  
- **attendance_percentage** – Attendance rate (%)  
- **sleep_hours** – Average sleep per day  
- **social_media_hours** – Daily time spent on social media  

These variables allow us to test whether AI usage interacts with traditional academic behaviors.

---

## Performance Metrics  

Additional academic measures include:

- **last_exam_score** – Most recent exam performance  
- **assignment_scores_avg** – Average assignment score  
- **improvement_rate** – Rate of academic improvement over time  

These metrics help evaluate whether AI usage is associated with measurable academic growth.

---

## Dataset Limitations & Concerns  

### Self-Reported Measures  
Several variables (AI usage time, sleep hours, social media hours, concept understanding) may be self-reported and subject to reporting bias.

### Categorical Encoding  
Some variables are categorical but may be numerically encoded (e.g., `uses_ai`, `passed`). These must be treated appropriately during modeling to avoid incorrect linear assumptions.

### Potential Target Leakage  
Variables such as `passed` and `performance_category` are derived from academic scores and should not be included as predictors when modeling `final_score`.

### Cross-Sectional Structure  
The dataset appears to represent a single time snapshot rather than longitudinal tracking, limiting causal inference.

### AI Usage Confounding  
Students who use AI may differ systematically (e.g., motivation, socioeconomic status), which may confound performance outcomes.

---

## Research Potential  

This dataset allows exploration of questions such as:

- Does AI usage improve academic performance?
- Is heavy AI dependency associated with weaker conceptual understanding?
- Does AI usage substitute or complement traditional study habits?
- Are there differences in AI effectiveness across demographic groups?


In [ ]:
df2 = pd.read_csv('data/00-raw/ai-impact-students-performance-dataset.csv')
df2.head()

,student_id,age,gender,grade_level,study_hours_per_day,uses_ai,ai_usage_time_minutes,ai_tools_used,ai_usage_purpose,ai_dependency_score,...,concept_understanding_score,study_consistency_index,improvement_rate,sleep_hours,social_media_hours,tutoring_hours,class_participation_score,final_score,passed,performance_category
0,1,20,Female,1st Year,2.5,1,170,NaN,Exam Prep,10,...,4,9.0,8.1,7.2,1.5,4.6,6,36.8,0,Low
1,2,17,Male,12th,3.4,1,123,NaN,Notes,4,...,8,8.5,13.8,7.6,5.1,2.7,6,65.5,1,Medium
2,3,24,Male,3rd Year,0.8,0,35,Copilot,Doubt Solving,8,...,7,2.1,39.1,5.7,0.2,4.0,3,66.3,1,Medium
3,4,21,Female,12th,4.4,0,45,ChatGPT+Gemini,Notes,3,...,5,6.7,25.2,8.2,4.2,2.9,2,69.5,1,Medium
4,5,18,Other,3rd Year,3.5,1,21,ChatGPT+Gemini,Coding,2,...,5,5.4,2.7,8.7,0.2,2.9,2,49.7,1,Low


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   student_id                       8000 non-null   int64  
 1   age                              8000 non-null   int64  
 2   gender                           8000 non-null   object 
 3   grade_level                      8000 non-null   object 
 4   study_hours_per_day              8000 non-null   float64
 5   uses_ai                          8000 non-null   int64  
 6   ai_usage_time_minutes            8000 non-null   int64  
 7   ai_tools_used                    6638 non-null   object 
 8   ai_usage_purpose                 6654 non-null   object 
 9   ai_dependency_score              8000 non-null   int64  
 10  ai_generated_content_percentage  8000 non-null   int64  
 11  ai_prompts_per_week              8000 non-null   int64  
 12  ai_ethics_score     

In [ ]:
df2.columns

Index(['student_id', 'age', 'gender', 'grade_level', 'study_hours_per_day',
       'uses_ai', 'ai_usage_time_minutes', 'ai_tools_used', 'ai_usage_purpose',
       'ai_dependency_score', 'ai_generated_content_percentage',
       'ai_prompts_per_week', 'ai_ethics_score', 'last_exam_score',
       'assignment_scores_avg', 'attendance_percentage',
       'concept_understanding_score', 'study_consistency_index',
       'improvement_rate', 'sleep_hours', 'social_media_hours',
       'tutoring_hours', 'class_participation_score', 'final_score', 'passed',
       'performance_category'],
      dtype='object')

In [ ]:
df2.shape # 8000 rows and 26 columns

(8000, 26)

In [ ]:
df2.isna().sum()

student_id                            0
age                                   0
gender                                0
grade_level                           0
study_hours_per_day                   0
uses_ai                               0
ai_usage_time_minutes                 0
ai_tools_used                      1362
ai_usage_purpose                   1346
ai_dependency_score                   0
ai_generated_content_percentage       0
ai_prompts_per_week                   0
ai_ethics_score                       0
last_exam_score                       0
assignment_scores_avg                 0
attendance_percentage                 0
concept_understanding_score           0
study_consistency_index               0
improvement_rate                      0
sleep_hours                           0
social_media_hours                    0
tutoring_hours                        0
class_participation_score             0
final_score                           0
passed                                0


Missing data is only on ai_tools_used and ai_usage_purpose which are empty if ai tools are not used by the students, because of that we will drop those columns, we also find that it won't really help our main argument

In [ ]:
df2_clean = df2.drop(columns=['ai_tools_used','ai_usage_purpose'])
df2_clean.head()

,student_id,age,gender,grade_level,study_hours_per_day,uses_ai,ai_usage_time_minutes,ai_dependency_score,ai_generated_content_percentage,ai_prompts_per_week,...,concept_understanding_score,study_consistency_index,improvement_rate,sleep_hours,social_media_hours,tutoring_hours,class_participation_score,final_score,passed,performance_category
0,1,20,Female,1st Year,2.5,1,170,10,29,56,...,4,9.0,8.1,7.2,1.5,4.6,6,36.8,0,Low
1,2,17,Male,12th,3.4,1,123,4,14,65,...,8,8.5,13.8,7.6,5.1,2.7,6,65.5,1,Medium
2,3,24,Male,3rd Year,0.8,0,35,8,82,8,...,7,2.1,39.1,5.7,0.2,4.0,3,66.3,1,Medium
3,4,21,Female,12th,4.4,0,45,3,74,106,...,5,6.7,25.2,8.2,4.2,2.9,2,69.5,1,Medium
4,5,18,Other,3rd Year,3.5,1,21,2,83,7,...,5,5.4,2.7,8.7,0.2,2.9,2,49.7,1,Low


In [ ]:
df2_clean.describe()

,student_id,age,study_hours_per_day,uses_ai,ai_usage_time_minutes,ai_dependency_score,ai_generated_content_percentage,ai_prompts_per_week,ai_ethics_score,last_exam_score,...,attendance_percentage,concept_understanding_score,study_consistency_index,improvement_rate,sleep_hours,social_media_hours,tutoring_hours,class_participation_score,final_score,passed
count,8000.00000,8000.00000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,...,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,4000.50000,18.96725,3.286938,0.641000,88.989875,5.515125,50.057500,59.359500,5.472375,59.501250,...,69.852112,5.484750,5.516450,10.127325,6.465025,3.010600,2.523387,5.463375,56.811988,0.889250
std,2309.54541,3.15316,1.582171,0.479737,52.184077,2.883064,29.440109,34.869496,2.872067,23.161888,...,17.234339,2.898546,2.586377,17.479288,1.435670,1.744974,1.436991,2.881917,13.455724,0.313842
min,1.00000,14.00000,0.500000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,20.000000,...,40.000000,1.000000,1.000000,-20.000000,4.000000,0.000000,0.000000,1.000000,12.700000,0.000000
25%,2000.75000,16.00000,1.900000,0.000000,43.000000,3.000000,25.000000,29.000000,3.000000,39.750000,...,54.800000,3.000000,3.200000,-5.000000,5.200000,1.500000,1.300000,3.000000,47.300000,1.000000
50%,4000.50000,19.00000,3.300000,1.000000,89.000000,6.000000,50.000000,59.000000,6.000000,59.000000,...,70.000000,5.000000,5.500000,10.150000,6.400000,3.000000,2.500000,5.000000,56.900000,1.000000
75%,6000.25000,22.00000,4.600000,1.000000,134.000000,8.000000,76.000000,90.000000,8.000000,80.000000,...,84.600000,8.000000,7.800000,25.500000,7.700000,4.500000,3.800000,8.000000,66.200000,1.000000
max,8000.00000,24.00000,6.000000,1.000000,179.000000,10.000000,100.000000,119.000000,10.000000,99.000000,...,100.000000,10.000000,10.000000,40.000000,9.000000,6.000000,5.000000,10.000000,95.800000,1.000000


With this the dataset is clean and is ready to use for our purpose.

In [ ]:
df2_clean.to_csv('data/01-interim/df2_clean.csv', index=False)

## Ethics

## Ethics

Our team has carefully considered ethical implications across the full lifecycle of this project, including data collection, storage, analysis, modeling, and potential deployment.

---

### A. Data Collection

#### A.1 Informed Consent — Considered ✓
We will only use publicly available datasets that were collected with appropriate consent for academic research. Dataset documentation will be reviewed to confirm ethical collection practices before any data is used.

#### A.2 Collection Bias — Considered ✓
Survey-based student datasets may contain self-selection bias. Students who choose to participate may differ from those who do not. We acknowledge that this may limit generalizability and will clearly state this limitation in our report.

#### A.3 Limit PII Exposure — Considered ✓
We will only use datasets that are anonymized and do not include personally identifiable information (PII). Identity-level data such as names, emails, or IDs are not necessary for our analysis and will not be used.

#### A.4 Downstream Bias Mitigation — Considered ✓
If demographic variables are included, we will use them carefully to test for fairness rather than to reinforce stereotypes.

---

### B. Data Storage

#### B.1 Data Security — Considered ✓
We will not share raw data beyond what is publicly available from the original source.

#### B.2 Right to Be Forgotten — Considered ✓
Because we are not collecting new data and will only use anonymized public datasets, individual removal requests are not applicable to this project.

#### B.3 Data Retention Plan — Considered ✓
The dataset will only be used for the duration of this course project. We do not plan to retain or reuse the data beyond academic purposes.

---

### C. Analysis

#### C.1 Missing Perspectives — Considered ✓
We recognize that our team may not represent all student experiences. We rely on prior research and will clearly state limitations in interpretation.

#### C.2 Dataset Bias — Considered ✓
We will examine variable distributions and discuss possible biases and potential confounding factors.

#### C.3 Honest Representation — Considered ✓
All visualizations and statistical summaries will accurately represent the data. We will avoid misleading scales and clearly label all figures.

#### C.4 Privacy in Analysis — Considered ✓
We will not display or analyze any personally identifiable information.

#### C.5 Auditability — Considered ✓
All data cleaning and modeling steps will be documented in our notebook to ensure transparency and reproducibility.

---

### D. Modeling

#### D.1 Proxy Discrimination — Considered ✓
We will examine whether any variables may unintentionally act as proxies for protected characteristics and interpret results carefully.

#### D.2 Fairness Across Groups — Considered ✓
We will evaluate whether predictive performance differs across demographic groups.

#### D.3 Metric Selection — Considered ✓
We will use appropriate regression metrics and clearly explain why they were selected.

#### D.4 Explainability — Considered ✓
We will use interpretable methods and explain results in accessible language.

#### D.5 Communicate Limitations — Considered ✓
We will clearly state that our findings demonstrate associations rather than causation, and we will discuss dataset and modeling limitations.

---

### E. Deployment

#### E.1 Monitoring and Evaluation — Considered ✓
This is an academic project and will not be deployed in a real-world setting.

#### E.2 Redress — Considered ✓
Since the model will not be used for real decision-making, direct harm is unlikely.

#### E.3 Roll Back — Considered ✓
The model exists only within our project notebook and can be easily modified or removed.

#### E.4 Unintended Use — Considered ✓
We will clearly state that this model is for educational purposes only and should not be used for high-stakes decisions about students.


## Team Expectations 

Communication: 
- We will meet once per week to discuss progress and assign tasks. 
- We communicate primarily through Discord.
- Members will respond to messages within 24 hours unless they inform the group ahead of time. 
- Important decisions will be written in our group chat, shared document or GitHub repo. 

Work Distribution: 
- Tasks will be divided based on strengths and interests (data cleaning, EDA, modeling, writing, visuals).
- Every member will contribute to both coding and writing. 
- Darts must be shared at least 48 hours before deadlines so everyone can review. 
- No section will be submitted without full group agreement. 

Accountability: 
- Each member is responsible for completing assigned tasks by the agreed deadline. 
- If someone cannot complete their work, they must inform the group as soon as possible.
- If participation becomes uneven, we will address it respectfully and early. 

Conflict Resolution:
- We agree to communicate respectfully, even when we disagree
- If conflicts arise, we will first try to resolve them as a group. 
- If necessary, we will consult a TA for support. 

GitHub & Collaboration: 
- Changes will be clearly labeled with descriptive commit messages. 
- We will not overwrite each other’s work without discussion. 

Academic Integrity: 
- All code and writing will be properly cited. 
- We will not copy from other groups or online submissions. 



## Project Timeline Proposal

Instructions: Replace this with your timeline.  **PLEASE UPDATE your Timeline!** No battle plan survives contact with the enemy, so make sure we understand how your plans have changed.  Also if you have lost points on the previous checkpoint fix them